In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import argparse
import os
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch
from functools import partial
from fastai.vision import *
import fastai.vision
from sklearn.metrics import f1_score
import os
from PIL import Image
from sklearn.metrics import precision_score

from torchvision.models import *
import pretrainedmodels

from fastai.vision.models import *
from fastai.vision.learner import model_meta
from fastai.vision.gan import GANLearner
import cv2
from tqdm import tqdm_notebook
import tqdm

## Images Path

In [3]:
PATH = '../Data/game_of_deep_learning/'

In [4]:
train_files = pd.read_csv(os.path.join(PATH,'train.csv'))

In [13]:
#Count of images per category
train_files.category.value_counts()

1    2120
5    1217
2    1167
3     916
4     832
Name: category, dtype: int64

# Data Transform Fastai

In [50]:
img_size = 320
batch_size = 20

xtra_tfm = [
    brightness(change=[0.3, 0.5]),
    contrast(scale=[1.5, 0.5]),
    perspective_warp(magnitude=[0.5, 0.7])
]

tfms = get_transforms(xtra_tfms=xtra_tfm)

train_data = (ImageList.from_csv(
    path_img, 'train.csv', folder='images',
    cols='image').split_none().label_from_df(cols='category').transform(
        tfms, size=img_size, resize_method=ResizeMethod.SQUISH).databunch(
            bs=batch_size).normalize(imagenet_stats))

test_data = ImageList.from_csv(path_img, 'test_ApKoW4T.csv', folder='images')

train_data.add_test(test_data)

# ResneXt

In [ ]:
def resnext101_32(pretrained = False):
    pretrained = 'imagenet' if pretrained else None
    model = pretrainedmodels.resnext101_64x4d(pretrained = pretrained)
    all_layers = list(model.children())
    return nn.Sequential(*all_layers[0], *all_layers[1:])

In [51]:
def get_model(pretrained = False):
    pretrained = 'imagenet' if pretrained else None
    model = pretrainedmodels.se_resnext101_32x4d(pretrained = pretrained)
    all_layers = list(model.children())
    return nn.Sequential(*all_layers[0], *all_layers[1:])

In [62]:
def nas_net(pretrained = False):
    pretrained = 'imagenet' if pretrained else None
    model = pretrainedmodels.nasnetalarge(num_classes = 1000,pretrained = pretrained)
    all_layers = list(model.children())
    return nn.Sequential(*all_layers[0], *all_layers[1:])

In [ ]:
# learn = cnn_learner(train_data, nas_net, pretrained = True,
#                   cut = -2, split_on = lambda m:(m[0][6],m[1]))
learn = cnn_learner(train_data, nas_net, pretrained = True,cut = -2)

In [ ]:
train_data.show_batch(rows = 3, figsize = (10,10))

In [ ]:
learn.freeze()
learn.fit_one_cycle(25,max_lr = 1e-3)

In [ ]:
learn.save('resnext101_game_of_deep_learning_v1')

# Inception

In [ ]:
def inceptionv4(pretrained = False):
    pretrained = 'imagenet' if pretrained else None
    model = pretrainedmodels.inceptionv4(pretrained = pretrained)
    all_layers = list(model.children())
    return nn.Sequential(*all_layers[0], *all_layers[1:])


In [ ]:
learn = cnn_learner(train_data,inceptionv4,pretrained = True,
                   cut=-2, split_on = lambda m:(m[0][11], m[1]))

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(10,max_lr = (1e-3,1e-2,1e-1))

In [ ]:
learn.show_results(rows = 3, figsize = (12,15))

# Interpretation

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_confusion_matrix()

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(15,figsize = (15,11))

In [ ]:
valid_preds, valid_labels = learn.get_preds()

In [ ]:
valid_preds = np.argmax(valid_preds,1)
print(f1_score(valid_labels, valid_preds,average= 'weighted'))

# SMRT

In [ ]:
from smrt.autoencode import AutoEncoder

# GAN

In [ ]:
img_size = 224
lr = 1e-3
b1 = 0.5
b2 = 0.999
batch_size = 64
latent_dim = 100
channels = 3
sample_interval = 400
n_epochs = 200
nz = latent_dim
ngf = 64
ndf = 64
nc = channels

In [ ]:
train_files[train_files.category == 5].reset_index(drop = True).to_csv('./category_5.csv', index = False)

In [ ]:
class deep_learning_data(Dataset):
    
    def __init__(self, csv_file, root_dir, transform = None,y=True):
        
        self.root_dir = root_dir
        self.transform = transform
        self.files = pd.read_csv(csv_file)
        self.y = y
        
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,
                               self.files.iloc[idx,0])
        image = Image.fromarray(cv2.imread(img_name))
        if self.transform:
            image = self.transform(image)
        if (np.array(image.size()) == np.array([1,img_size,img_size])).all():
            print(img_name)

        return (image, self.files.iloc[idx,1]-1)
        
img_transforms = {
    'train': 
    transforms.Compose([
        transforms.Resize(size =(img_size,img_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
    ]),
    
    'valid':
    transforms.Compose([
        transforms.Resize(size=(img_size,img_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
    ])
}

train_dataset= deep_learning_data(csv_file = './category_5.csv',
                                 root_dir = '../data/train/images',
                                 transform = img_transforms['train'])

In [ ]:
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d(ngf * 2,ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d(ngf,nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)
        return output



In [ ]:
cuda = True if torch.cuda.is_available() else False


def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        if input.is_cuda and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)

        return output


# Loss function
adversarial_loss = torch.nn.BCELoss()

# Initialize generator and discriminator
generator = Generator(1)
discriminator = Discriminator(1)

if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

# Initialize weights
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)


In [ ]:
dataloader = DataLoader(train_dataset,batch_size = batch_size,
                       shuffle = True, num_workers = 4)

# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(b1, b2))
criterion = nn.BCELoss()
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [ ]:
device = torch.device("cuda:0")
real_label = 1
fake_label = 0
fixed_noise = torch.randn(batch_size, nz, 1,1,device =device)

In [ ]:
output.shape,label.shape,real_cpu.shape

In [ ]:
t = tqdm_notebook(range(n_epochs))

for epoch in t:
    for i, data in enumerate(dataloader, 0):
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        # train with real
        discriminator.zero_grad()
        real_cpu = data[0].to(device)
        batch_size = real_cpu.size(0)
        label = torch.full((batch_size,), real_label, device=device)

        output = discriminator(real_cpu)
        errD_real = criterion(output, label)
        errD_real.backward()
        D_x = output.mean().item()

        # train with fake
        noise = torch.randn(batch_size, nz, 1, 1, device=device)
        fake = generator(noise)
        label.fill_(fake_label)
        output = discriminator(fake.detach())
        errD_fake = criterion(output, label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        generator.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        output = discriminator(fake)
        errG = criterion(output, label)
        errG.backward()
        D_G_z2 = output.mean().item()
        optimizerG.step()

        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f'
              % (epoch, opt.niter, i, len(dataloader),
                 errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))
        if i % 100 == 0:
            vutils.save_image(real_cpu,
                    '%s/real_samples.png' % opt.outf,
                    normalize=True)
            fake = generator(fixed_noise)
            vutils.save_image(fake.detach(),
                    '%s/fake_samples_epoch_%03d.png' % (opt.outf, epoch),
                    normalize=True)

# EfficientNet

In [ ]:
from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_pretrained('efficientnet-b3')

In [ ]:
optimizer = optim.SGD(model.parameters(), lr= 0.001, momentum = 0.9)
criterion = nn.CrossEntropyLoss()
model = model.cuda()

In [ ]:
n_epochs = 20

val_acc_history = []

# best_model_wts = copy.deepcopy(model.cpu().state_dict())

best_acc = 0.0
t = tqdm.tqdm_notebook(range(n_epochs))
k = tqdm.tqdm_notebook(train_dataloader)
for epoch in t:
    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in k:
#             pdb.set_trace()
#             print(inputs.shape)
        inputs = inputs.cuda()
        labels = labels.cuda()

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        _, preds = torch.max(outputs,1)

        loss.backward()
        optimizer.step()

    running_loss += loss.item() * inputs.size(0)
    running_corrects += torch.sum(preds == labels.data)
        
    epoch_loss = running_loss / len(train_dataloader.dataset)
    epoch_acc = running_corrects.double() / len(train_dataloader.dataset)
    
    print('{} Loss : {:.4f} Acc: {:.4f}'.format('Train',epoch_loss,epoch_acc))

# model.load_state_dict(best_model_wts)

In [ ]:
outputs[0],labels

# Submission

In [54]:
preds,_ = learn.TTA(ds_type = DatasetType.Test)

preds = np.argmax(preds,1)

In [ ]:
preds.shape

In [55]:
preds = preds+1

In [56]:
test_file = pd.read_csv('../Data/game_of_deep_learning/test_ApKoW4T.csv')

In [57]:
test_file.loc[:,'category'] = np.array(preds)

In [58]:
test_file.to_csv('submission_9.csv',index= False)